In [ ]:
import tensorflow as tf
from imblearn.over_sampling import SMOTE
import numpy as np
import tf2onnx
import onnx

from lhrNetUtils.data_loader import lhrNetDataLoader
from lhrNetUtils import Config

config = Config("config.json")

In [ ]:
n_states = len(config.get_states())

testDataLoader = lhrNetDataLoader("data/test_details.csv",config.get_y_length(),config.get_x_length(),n_states,8)
trainDataLoader = lhrNetDataLoader("data/train_details.csv",config.get_y_length(),config.get_x_length(),n_states,8)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((config.get_y_length(),config.get_x_length())),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64,activation="relu"),
    tf.keras.layers.Dense(n_states,activation="softmax")
])

loss_fn = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer="adam",loss=loss_fn,metrics=["accuracy"])

In [ ]:
oversample = SMOTE()

y = np.array(trainDataLoader.y)
flatX = np.reshape(trainDataLoader.X,(len(y),config.get_x_length()*config.get_y_length()))
flatX,y = oversample.fit_resample(flatX,y)

X = np.reshape(flatX,(len(y),config.get_y_length(),config.get_x_length()))
X = X.astype(np.float32)

In [ ]:
model.fit(x=X,y=y,batch_size=trainDataLoader.batch_size,epochs=60)


In [ ]:
import numpy as np
from sklearn.metrics import classification_report
y_pred = np.argmax(model.predict(testDataLoader), axis=-1)
print(classification_report(testDataLoader.y_arr, y_pred))

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
model.save("data/model.keras")

In [ ]:
model.output_names=["output"]
spec = (tf.TensorSpec((None, 32, 64), tf.float32, name="input"),)
onnx_model,_ = tf2onnx.convert.from_keras(model,input_signature=spec)

In [ ]:
onnx.save(onnx_model,"data/model.onnx")

In [ ]:
print(model.input_shape)
print([node.name for node in onnx_model.graph.output])